# 데이터 수집 및 가공 pt.2

#### CUAI 하계컨퍼런스용

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

# 주가 변동 데이터 구하기

참고: [주식 데이터 가져오기 관련; read_html()을 쓰는군](https://ai-creator.tistory.com/51)

## 1. 종목 코드 스크레이핑

#### 코스피(KOSPI)와 코스닥(KOSDAQ)의 종목 코드 목록은 [한국 거래소 사이트](https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage)에서 제공한다. 여기서 종목 코드를 스크레이핑 해오자.

In [2]:
#해당 링크는 한국거래소에서 상장법인목록을 엑셀로 다운로드하는 링크입니다.
#다운로드와 동시에 Pandas에 excel 파일이 load가 되는 구조입니다.
stock_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0] 

# 필요한 것은 "회사명"과 "종목코드" 이므로 필요없는 column들은 제외
stock_code = stock_code[['회사명', '종목코드']] 

# 한글 컬럼명을 영어로 변경 
stock_code = stock_code.rename(columns={'회사명': 'company', '종목코드': 'code'}) 
#stock_code.head()

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
stock_code.code = stock_code.code.map('{:06d}'.format) 

In [3]:
stock_code[stock_code['company'] == '삼성전자']

,company,code
946,삼성전자,005930


## 2. 네이버 금융 페이지에서 주식 정보 스크레이핑

#### [user-agent값을 확인하자; read_html()을 위한 준비 작업인듯](http://www.useragentstring.com/index.php)

참고: 
"다른 것들은 특별히 설명할 필요는 없을 것 같아 header에 추가된 User-Agent 만 설명 드리겠습니다.
 
네이버는 자신의 데이터를 가져가는 것을 원하지 않기 때문에, 프로그램으로 접근하는 것을 막고 있습니다.
 
그래서, 일반 사용자인척을 하기 위해 header에 제가 사용하는 브라우저의 User-Agent 를 추가해 주었습니다."

### 관심 주식

In [4]:
stocks = ['한전기술','셀트리온','카카오게임즈','iMBC','하이브',
          '삼성전자','현대자동차','HMM','대한항공','NAVER','두산중공업','SK하이닉스']  
# 한국 거래소에 현대차가 현대자동차로, 네이버가 NAVER로, sk하이닉스가 SK하이닉스로 나와있다.

### 테스트

In [5]:
import requests

# LG화학의 일별 시세 url 가져오기 
company = stocks[0]
code = stock_code[stock_code.company==company].code.values[0].strip() ## strip() : 공백제거
page = 1

url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)     
url = '{url}&page={page}'.format(url=url, page=page)

# read_html()용
header = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15'}  
res = requests.get(url,headers=header)

df = pd.read_html(res.text, header=0)[0]
df

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.07.16,53500.0,2000.0,55700.0,55800.0,53400.0,595269.0
2,2021.07.15,55500.0,2700.0,53500.0,57300.0,53100.0,1410497.0
3,2021.07.14,52800.0,600.0,53700.0,53800.0,52300.0,370598.0
4,2021.07.13,53400.0,700.0,54700.0,56300.0,52500.0,628112.0
5,2021.07.12,54100.0,1900.0,58000.0,58400.0,53900.0,846709.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021.07.09,56000.0,3400.0,51700.0,57000.0,51300.0,1606667.0


#### 불러온 데이터의 결측치를 제거하면서 저장해야겠다

In [6]:
from tqdm import tqdm  #tqdm은 for문의 진행 정도를 시각화한다

data2 = pd.DataFrame() # 실제 데이터 저장용

for stock in tqdm(stocks):
    code = stock_code[stock_code.company==stock].code.values[0].strip() ## strip() : 공백제거

    for page in range(1,15):  # 페이지 14에 2021-01-04 로 시작해 페이지 1에 2021-07-15의 데이터가 있다
        temp = pd.DataFrame() # 위를 보면 불러온 데이터에 결측값이 포함된 행들이 있다. 이를 제거해주기 위해 임시로 이 df에다가 저장한다
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)     
        url = '{url}&page={page}'.format(url=url, page=page)
        res = requests.get(url,headers=header)
        temp = pd.read_html(res.text, header=0)[0]
        temp = temp.dropna(axis=0)  # 결측치 제거
        temp['주식'] = stock
        data2 = data2.append(temp)

100%|██████████| 12/12 [00:19<00:00,  1.61s/it]


In [7]:
data2

,날짜,종가,전일비,시가,고가,저가,거래량,주식
1,2021.07.16,53500.0,2000.0,55700.0,55800.0,53400.0,595269.0,한전기술
2,2021.07.15,55500.0,2700.0,53500.0,57300.0,53100.0,1410497.0,한전기술
3,2021.07.14,52800.0,600.0,53700.0,53800.0,52300.0,370598.0,한전기술
4,2021.07.13,53400.0,700.0,54700.0,56300.0,52500.0,628112.0,한전기술
5,2021.07.12,54100.0,1900.0,58000.0,58400.0,53900.0,846709.0,한전기술
...,...,...,...,...,...,...,...,...
9,2020.12.30,118500.0,2500.0,116000.0,118500.0,114500.0,4195192.0,SK하이닉스
10,2020.12.29,116000.0,500.0,118000.0,118500.0,114000.0,4208916.0,SK하이닉스
11,2020.12.28,115500.0,2500.0,119000.0,119500.0,115000.0,4825132.0,SK하이닉스
12,2020.12.24,118000.0,2000.0,116000.0,118500.0,115500.0,3486636.0,SK하이닉스


## 3. 데이터 클렌징

#### 전일비(음수, 양수인지 구분할 수 없음), 시가, 고가, 저가는 필요가 없어보인다.

In [8]:
data2 = data2.drop(['전일비','시가','고가','저가'], axis=1)

In [9]:
data2

,날짜,종가,거래량,주식
1,2021.07.16,53500.0,595269.0,한전기술
2,2021.07.15,55500.0,1410497.0,한전기술
3,2021.07.14,52800.0,370598.0,한전기술
4,2021.07.13,53400.0,628112.0,한전기술
5,2021.07.12,54100.0,846709.0,한전기술
...,...,...,...,...
9,2020.12.30,118500.0,4195192.0,SK하이닉스
10,2020.12.29,116000.0,4208916.0,SK하이닉스
11,2020.12.28,115500.0,4825132.0,SK하이닉스
12,2020.12.24,118000.0,3486636.0,SK하이닉스


In [10]:
# 날짜의 타입을 date로 바꿔줌 
data2['날짜'] = pd.to_datetime(data2['날짜']) 

In [11]:
data2

,날짜,종가,거래량,주식
1,2021-07-16,53500.0,595269.0,한전기술
2,2021-07-15,55500.0,1410497.0,한전기술
3,2021-07-14,52800.0,370598.0,한전기술
4,2021-07-13,53400.0,628112.0,한전기술
5,2021-07-12,54100.0,846709.0,한전기술
...,...,...,...,...
9,2020-12-30,118500.0,4195192.0,SK하이닉스
10,2020-12-29,116000.0,4208916.0,SK하이닉스
11,2020-12-28,115500.0,4825132.0,SK하이닉스
12,2020-12-24,118000.0,3486636.0,SK하이닉스


### 현재 엉망인 인덱스를 주식마다 0-139로 인덱스를 갖도록 만들어보자

In [12]:
data2 = data2.reset_index(drop=True)

In [13]:
data2['idx'] = 0 # 초기화

In [14]:
for i in range(len(data2)):
    data2['idx'][i] = i % 140

data2

,날짜,종가,거래량,주식,idx
0,2021-07-16,53500.0,595269.0,한전기술,0
1,2021-07-15,55500.0,1410497.0,한전기술,1
2,2021-07-14,52800.0,370598.0,한전기술,2
3,2021-07-13,53400.0,628112.0,한전기술,3
4,2021-07-12,54100.0,846709.0,한전기술,4
...,...,...,...,...,...
1675,2020-12-30,118500.0,4195192.0,SK하이닉스,135
1676,2020-12-29,116000.0,4208916.0,SK하이닉스,136
1677,2020-12-28,115500.0,4825132.0,SK하이닉스,137
1678,2020-12-24,118000.0,3486636.0,SK하이닉스,138


In [15]:
data2 = data2.set_index('idx')

In [16]:
data2

,날짜,종가,거래량,주식
idx,,,,
0,2021-07-16,53500.0,595269.0,한전기술
1,2021-07-15,55500.0,1410497.0,한전기술
2,2021-07-14,52800.0,370598.0,한전기술
3,2021-07-13,53400.0,628112.0,한전기술
4,2021-07-12,54100.0,846709.0,한전기술
...,...,...,...,...
135,2020-12-30,118500.0,4195192.0,SK하이닉스
136,2020-12-29,116000.0,4208916.0,SK하이닉스
137,2020-12-28,115500.0,4825132.0,SK하이닉스


## 4. 주가변동여부, 거래량변동여부 열 만들기

#### 전일과 비교해서 종가가 작거나 같으면 0, 크면 1로 하자; 거래량도 똑같이 전일과 비교해서 작거나 같으면 0, 크면 1

In [17]:
data_stock = pd.DataFrame()  # 최종 데이터를 담을 데이터프레임 초기화

In [18]:
for stock in stocks:
    data_temp = data2[data2['주식']==stock]
    data_temp['주가변동여부'] = 0  # 주가변동여부 열 초기화
    data_temp['거래량변동여부'] = 0  # 거래량변동여부 열 초기화

    for i in range(len(data_temp)-1):  # -1을 한 이유는 마지막 행은 비교대상이 없기 때문
        if data_temp['종가'][i] > data_temp['종가'][i+1]:
            data_temp['주가변동여부'][i] = 1
        if data_temp['거래량'][i] > data_temp['거래량'][i+1]:
            data_temp['거래량변동여부'][i] = 1
            
    data_temp = data_temp.drop(index=139)  # 마지막 행은 i+1의 비교대상이 없으므로 정확한 주가변동여부가 아니기에 제거

    data_stock = data_stock.append(data_temp)

#### 주가변동여부를 만들었으므로 종가와 거래량은 더이상 필요가 없다

In [19]:
data_stock = data_stock.drop(['종가','거래량'], axis=1)

In [20]:
data_stock

,날짜,주식,주가변동여부,거래량변동여부
idx,,,,
0,2021-07-16,한전기술,0,0
1,2021-07-15,한전기술,1,1
2,2021-07-14,한전기술,0,0
3,2021-07-13,한전기술,0,0
4,2021-07-12,한전기술,0,0
...,...,...,...,...
134,2021-01-04,SK하이닉스,1,1
135,2020-12-30,SK하이닉스,1,0
136,2020-12-29,SK하이닉스,1,0


#### 총 12개의 주식에서 1개의 행씩 빠져서 총 1668개의 데이터가 생성

In [21]:
data_stock.to_csv('data_stock.csv', index=False)